Are college students dropping out?
What universities are doing "well"?

In [20]:
import polars as pl
import os
from pathlib import Path

In [35]:
data_dir_path = Path('data')

merged_pp_2021 = data_dir_path / 'data' / 'MERGED2021_22_PP.csv'
import pandas as pd

df = pd.read_csv(merged_pp_2021, chunksize=20)
df

In [30]:
df = df.drop(columns=[col for col in df.columns if col.startswith('CIP')])
df

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOPELL_FIRSTTIME,OMENRUP_NOPELL_FIRSTTIME,OMENRYP_NOPELL_NOTFIRSTTIME,OMENRAP_NOPELL_NOTFIRSTTIME,OMAWDP8_NOPELL_NOTFIRSTTIME,OMENRUP_NOPELL_NOTFIRSTTIME,OMACHT8_NOPELL_ALL,OMACHT8_NOPELL_FIRSTTIME,OMACHT8_NOPELL_NOTFIRSTTIME,ADDR
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,0.3187,0.2709,0.0128,0.2949,0.4744,0.2179,329.0,251.0,78.0,4900 Meridian Street
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,0.6937,0.0660,0.0111,0.2636,0.5136,0.2117,2358.0,1182.0,1176.0,Administration Bldg Suite 1070
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,0.0000,0.5000,0.0000,0.3333,0.4583,0.2083,26.0,2.0,24.0,1200 Taylor Rd
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,0.6471,0.0941,0.0082,0.2647,0.5948,0.1324,1122.0,510.0,612.0,301 Sparkman Dr
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,0.4381,0.2167,0.0000,0.1444,0.3667,0.4889,510.0,420.0,90.0,915 S Jackson Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538,49576720,332918.0,3329.0,Pennsylvania State University-Penn State York,York,PA,17403-3326,Middle States Commission on Higher Education,york.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6539,49576721,332919.0,3329.0,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,Middle States Commission on Higher Education,greatvalley.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6540,49576722,332920.0,3329.0,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,Middle States Commission on Higher Education,harrisburg.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6541,49576723,332921.0,3329.0,Pennsylvania State University-Penn State Brand...,Media,PA,19063-5522,Middle States Commission on Higher Education,brandywine.psu.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Hey Boss

Here is a dataframe with all colleges, cities, and states, along with their accrediting agency and SAT score:

In [31]:
small_df = df[['INSTNM', 'CITY', 'STABBR', 'ACCREDAGENCY', 'SAT_AVG_ALL', 'LATITUDE', 'LONGITUDE']]
small_df


,INSTNM,CITY,STABBR,ACCREDAGENCY,SAT_AVG_ALL,LATITUDE,LONGITUDE
0,Alabama A & M University,Normal,AL,Southern Association of Colleges and Schools C...,954.0,34.783368,-86.568502
1,University of Alabama at Birmingham,Birmingham,AL,Southern Association of Colleges and Schools C...,1266.0,33.505697,-86.799345
2,Amridge University,Montgomery,AL,Southern Association of Colleges and Schools C...,NaN,32.362609,-86.174010
3,University of Alabama in Huntsville,Huntsville,AL,Southern Association of Colleges and Schools C...,1300.0,34.724557,-86.640449
4,Alabama State University,Montgomery,AL,Southern Association of Colleges and Schools C...,955.0,32.364317,-86.295677
...,...,...,...,...,...,...,...
6538,Pennsylvania State University-Penn State York,York,PA,Middle States Commission on Higher Education,1260.0,NaN,NaN
6539,Pennsylvania State University-Penn State Great...,Malvern,PA,Middle States Commission on Higher Education,1260.0,NaN,NaN
6540,Pennsylvania State University-Penn State Harri...,Middletown,PA,Middle States Commission on Higher Education,1260.0,NaN,NaN
6541,Pennsylvania State University-Penn State Brand...,Media,PA,Middle States Commission on Higher Education,1260.0,NaN,NaN


Not all universities have an SAT score, so we'll only consider those that do:

In [32]:
small_df = small_df.dropna(subset=['SAT_AVG_ALL', 'LATITUDE', 'LONGITUDE'])
small_df

,INSTNM,CITY,STABBR,ACCREDAGENCY,SAT_AVG_ALL,LATITUDE,LONGITUDE
0,Alabama A & M University,Normal,AL,Southern Association of Colleges and Schools C...,954.0,34.783368,-86.568502
1,University of Alabama at Birmingham,Birmingham,AL,Southern Association of Colleges and Schools C...,1266.0,33.505697,-86.799345
3,University of Alabama in Huntsville,Huntsville,AL,Southern Association of Colleges and Schools C...,1300.0,34.724557,-86.640449
4,Alabama State University,Montgomery,AL,Southern Association of Colleges and Schools C...,955.0,32.364317,-86.295677
5,The University of Alabama,Tuscaloosa,AL,Southern Association of Colleges and Schools C...,1244.0,33.211875,-87.545978
...,...,...,...,...,...,...,...
5935,Northern Pennsylvania Regional College,Warren,PA,NaN,1154.0,41.845727,-79.147078
5960,California Indian Nations College,Palm Desert,CA,NaN,1154.0,33.773732,-116.353365
5978,The Pennsylvania State University,University Park,PA,Middle States Commission on Higher Education,1260.0,40.796500,-77.862848
6000,Daybreak University,Anaheim,CA,NaN,1154.0,33.837857,-117.889540


In [33]:
small_df.columns

Index(['INSTNM', 'CITY', 'STABBR', 'ACCREDAGENCY', 'SAT_AVG_ALL', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [34]:
import pandas as pd
import plotly.express as px

# Assuming df is your DataFrame

# Create a scatter plot on a map
fig = px.scatter_geo(df,
                     lat='LATITUDE',
                     lon='LONGITUDE',
                     hover_name='INSTNM',  # Marker hover shows institution name
                     color='SAT_AVG_ALL',  # Colour of marker represents SAT_AVG_ALL
                     title='Colleges and Universities in the US',
                     scope='usa',  # Display only USA map
                     )

fig.show()


In [29]:

import plotly.express as px

# grouped DataFrame
grouped_df = df.groupby("STABBR")["SAT_AVG_ALL"].mean().reset_index()
sorted_df = grouped_df.sort_values("SAT_AVG_ALL", ascending=True)
# make a plotly figure
fig = px.bar(sorted_df, x='STABBR', y='SAT_AVG_ALL',
             title='Average SAT Scores by State',
             labels={'STABBR': 'State', 'SAT_AVG_ALL': 'Average SAT Score'}, log_y=True)

fig.show()
